In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

import os
import cv2
import string
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset

In [2]:
import os
home_path = os.environ['HOME']+'/work/licence-plate' # 输入下载到容器中的地址，如home_path=os.environ['HOME']+'/work'
data_path = home_path + '/data'
!mkdir -p $data_path

dataset_url='https://codecraft-2019.obs.cn-north-1.myhuaweicloud.com/data/train-data.zip'

!wget $dataset_url -O $data_path/train-data.zip -P $data_path 
import zipfile
dataset_file = data_path + '/train-data.zip'
zip = zipfile.ZipFile(dataset_file)
zip.extractall(data_path)
zip.close()
!rm -rf $dataset_file


--2019-04-18 09:21:25--  https://codecraft-2019.obs.cn-north-1.myhuaweicloud.com/data/train-data.zip
Resolving codecraft-2019.obs.cn-north-1.myhuaweicloud.com (codecraft-2019.obs.cn-north-1.myhuaweicloud.com)... 100.125.40.3, 100.125.40.34
Connecting to codecraft-2019.obs.cn-north-1.myhuaweicloud.com (codecraft-2019.obs.cn-north-1.myhuaweicloud.com)|100.125.40.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47549916 (45M) [application/zip]
Saving to: ‘/home/jovyan/work/licence-plate/data/train-data.zip’

train-data.zip      100%[===================>]  45.35M  44.8MB/s    in 1.0s    

2019-04-18 09:21:26 (44.8 MB/s) - ‘/home/jovyan/work/licence-plate/data/train-data.zip’ saved [47549916/47549916]



In [3]:
torch.manual_seed(2019)
index = {u'深':0, u'秦':1, u'京':2, u'海':3, u'成':4, u'南':5, u'杭':6, u'苏':7, u'松':8}
for i, ch in enumerate(string.digits):
    index[ch] = 9 + i 
for i, ch in enumerate(string.ascii_uppercase):
    index[ch] = 19 + i
chars = [0] * 45
for ch, idx in index.items():
    chars[idx] = ch

In [4]:
class LicensePlateDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.labels = open(os.path.join(root_dir, 'train-data-label.txt'), encoding='utf-8').readlines()
        self.labels = [line.split(',') for line in self.labels]
        self.labels = [[img.strip(), label.strip()] for img, label in self.labels]
        self.normalize = normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            self.normalize,
        ])

    def __len__(self):
        return len(self.labels)
 
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, 'train-data', self.labels[idx][1])
        with Image.open(img_name) as img:
            image = img.convert('RGB')
        image = self.transform(image)
        label = np.array([index[i] for i in self.labels[idx][0]])
        label = torch.from_numpy(label)
        #print(label)
        return image, label

In [5]:
# def train(train_loader, model, criterion, optimizer, epoch):
#     model.train()
#     loss100 = 0.0
#     for i, (input, target) in enumerate(train_loader):
#         output = model(input)
#         loss = criterion(output, target)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         loss100 += loss.item()
#         if i % 100 == 99:
#             print('[Epoch %d, Batch %5d] loss: %.3f' % (epoch + 1, i + 1, loss100 / 100))
#             loss100 = 0.0

In [6]:
def accuracy(output, target):
    #batch_size * num_class * num_label
    #bath_size * num_label
    num_label = 9
    hit = 0.0
    total = 0.0
    with torch.no_grad():
        batch_size = target.size(0)
        pred = torch.argmax(output, dim=1)
        hit = 1.0 * torch.sum(torch.eq(pred, target)).item()
        total = 1.0 * batch_size * num_label
    return 1.0 * hit / total

In [7]:
def validate(val_loader, model, criterion):
    model.eval()
    losses = 0.0
    acc = 0.0
    cnt = 0
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()
            cnt += 1
            output = model(input)
            output = output.view(-1, 45, 9)
            loss = criterion(output, target)
            losses += loss.item()
            acc += accuracy(output, target)
    print("loss : %f acc : %f" % (losses, acc / cnt))

In [8]:
lr = 0.001
batch_size = 128
epochs = 10
model = models.resnet50(num_classes=405)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr)
# optimizer = torch.optim.SGD(model.parameters(), lr,
#                                 momentum=0.9,
#                                 weight_decay=0.0001)
model.cuda()
criterion.cuda()
root_dir = 'work/licence-plate/data/'
train_dataset = LicensePlateDataset(root_dir)
val_dataset = LicensePlateDataset(root_dir)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0)

val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size, shuffle=False,
        num_workers=0)

In [ ]:
for epoch in range(epochs):
    #train(train_loader, model, criterion, optimizer, epoch)
    model.train()
    loss = 0.0
    acc = 0.0
    for i, (input, target) in enumerate(train_loader):
#         print(input.shape)
#         print(target.shape)
#         #print(input, target)
        input = input.cuda()
        target = target.cuda()
        output = model(input)
        output = output.view(-1, 45, 9)
        loss = criterion(output, target)
        #print(target[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss += loss.item()
        acc += accuracy(output, target)
        print('[Epoch %d, Batch %5d] loss: %.3f acc: %.3f' % (epoch + 1, i + 1, loss, acc))
        loss = 0.0
        acc = 0.0
#torch.save(model, 'work/model.pkl')
torch.save(model.state_dict(), 'work/model-resnet50.pth')

[Epoch 1, Batch     1] loss: 8.173 acc: 0.015
[Epoch 1, Batch     2] loss: 8.255 acc: 0.046
[Epoch 1, Batch     3] loss: 8.368 acc: 0.048
[Epoch 1, Batch     4] loss: 7.954 acc: 0.040
[Epoch 1, Batch     5] loss: 7.924 acc: 0.057
[Epoch 1, Batch     6] loss: 7.458 acc: 0.049
[Epoch 1, Batch     7] loss: 7.464 acc: 0.043
[Epoch 1, Batch     8] loss: 7.198 acc: 0.045
[Epoch 1, Batch     9] loss: 7.161 acc: 0.040
[Epoch 1, Batch    10] loss: 7.016 acc: 0.036
[Epoch 1, Batch    11] loss: 6.842 acc: 0.053
[Epoch 1, Batch    12] loss: 6.947 acc: 0.042
[Epoch 1, Batch    13] loss: 6.951 acc: 0.054
[Epoch 1, Batch    14] loss: 6.897 acc: 0.032
[Epoch 1, Batch    15] loss: 6.842 acc: 0.043
[Epoch 1, Batch    16] loss: 7.131 acc: 0.049
[Epoch 1, Batch    17] loss: 6.852 acc: 0.042
[Epoch 1, Batch    18] loss: 6.855 acc: 0.042
[Epoch 1, Batch    19] loss: 6.740 acc: 0.057
[Epoch 1, Batch    20] loss: 6.825 acc: 0.041
[Epoch 1, Batch    21] loss: 6.759 acc: 0.051
[Epoch 1, Batch    22] loss: 6.810

In [ ]:
validate(val_loader, model, criterion)

In [ ]:
image_name = 'work/licence-plate/data/train-data/2c9ec2debce77459.jpg'
image = np.asarray(bytearray(open(image_name, 'rb').read()), dtype="uint8")

bin_file = cv2.imdecode(image, cv2.IMREAD_COLOR)

img = cv2.imread(image_name)
#plt.imshow(img)
plt.imshow(bin_file)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize,
        ])
with Image.open(image_name) as img:
   image = img.convert('RGB')
    
# image = Image.fromarray(cv2.cvtColor(bin_file,cv2.COLOR_BGR2RGB))  
# image = image.convert('RGB')

image = transform(image)
image = image[None]
inputs = image.cuda()
output = model(inputs)
output = output.view(45, 9)
mm = torch.argmax(output, dim=0)
print([chars[i] for i in mm])

In [ ]:
# resnet50 = models.resnet50(pretrained=True)
# !ls work

In [ ]:
# img = cv2.imread('2b6b3180b74c55b0.jpg')
# img = cv2.resize(img, (224,224))
# plt.imshow(img)


# image = transforms.ToTensor()(img)
# image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
# image = image[None]
# inputs = Variable(image)
# output = resnet50(inputs)
# torch.argmax(output)

In [ ]:
# #修改分类的数量为2
# num_ftrs = resnet50.fc.in_features
# resnet50.fc = nn.Linear(num_ftrs, 44 * 9)


# img = cv2.imread('/bigdata/gemfield/github/data/val/1/img_0.jpg')
# img = cv2.resize(img, (224,224))
# image = transforms.ToTensor()(img)
# image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
# image = image[None]
# inputs = Variable(image.cuda())
# output = resnet50(inputs)

# outputs = torch.stack([nn.Softmax(dim=0)(i) for i in output])
# outputs = outputs.mean(0)
# p, preds = torch.max(outputs, 0)

# # tensor -> scaler
# print(p.data.cpu().item())
# print(preds.data.cpu().item())

In [ ]:
# model = torch.load(model.pkl')
# img = cv2.imread('/bigdata/gemfield/github/data/val/1/img_0.jpg')
# img = cv2.resize(img, (224,224))
# image = transforms.ToTensor()(img)
# image = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(image)
# image = image[None]
# inputs = Variable(image
# output = model(inputs)

In [ ]:
# torch.manual_seed(20)
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, 2,requires_grad=True)
# target = torch.empty(3, 2, dtype=torch.long).random_(5)
# print(input)
# print(target)
# output = loss(input, target)
# output.backward()
# print(output.item())
# output = loss(input[:,:,0], target[:,0])
# output.backward()
# print(output.item())
# output = loss(input[:,:,1], target[:,1])
# output.backward()
# print(output.item())
# print(target.size(0))
# mm = torch.argmax(input, dim=1)
# a = torch.eq(mm, target)
# print(a)
# print(target.size())
# accuracy(input, target)